In [25]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class BertHidden(nn.Module):
    def __init__(self):
        super(BertHidden, self).__init__()
        
        # initialize the BERT model and tokenizer
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        
        # add a linear layer to predict the output class
        # self.linear = nn.Linear(self.bert_model.config.hidden_size, num_classes)
        
    def forward(self, inputs):
        # tokenize each word in the input
        res= []

        for input in inputs:
            tokenized_inputs = [self.tokenizer.encode(inp, add_special_tokens=True) for inp in input]
            
            # pad the tokenized inputs to the same length
            max_len = max([len(inp) for inp in tokenized_inputs])
            padded_inputs = torch.zeros(len(input), max_len).long()
            for i, inp in enumerate(tokenized_inputs):
                padded_inputs[i, :len(inp)] = torch.tensor(inp)
            
            # pass the padded inputs through the BERT model
            outputs = self.bert_model(padded_inputs)
            
            # get the output of the [CLS] token
            pooled_output = outputs[1]
            res.append(pooled_output)

        output = torch.stack(res,dim=0)

        return output

        # pass the pooled output through the linear layer to get the predicted class
        logits = self.linear(pooled_output)
        
        return logits

In [26]:
bert = BertHidden()

# generate a list of input strings
inputs = [["This is the presentence input","This is the claim input", "This is the postsentence input"],["This is the zero input","This is the claim input", "This is the second input"]]
labels = [0,1,0]

# pass the inputs through the BertTokenized model
hidden_layer = bert(inputs)

hidden_layer

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[[-0.8594, -0.1659,  0.4316,  ..., -0.0480, -0.5564,  0.8697],
         [-0.7444, -0.1215,  0.7430,  ...,  0.4240, -0.5016,  0.7806],
         [-0.8532, -0.2060,  0.2519,  ..., -0.3518, -0.5712,  0.8669]],

        [[-0.8755, -0.1946,  0.2685,  ..., -0.1115, -0.5913,  0.8909],
         [-0.9099, -0.2416,  0.1094,  ..., -0.3110, -0.6026,  0.9285],
         [-0.7988, -0.1377,  0.6096,  ...,  0.1994, -0.5240,  0.8054]]],
       grad_fn=<StackBackward0>)

768